In [1]:
import pandas as pd
import time as t
from pathlib import Path
import xlwings as xw
import YM 
from shutil import copyfile

In [2]:
ts1 = t.time()

In [3]:
file_1c = t.strftime("%m%d") + "0900.xlsx"
file_path_1c = Path("D:/temp/__удалить/", file_1c)
file_path_cumulative = Path("D:/Analytik/1_Основное планирование/", "01_ИСХОДНИК-НОВ (выгрузка 1С).xlsx")
file_path_out_test = Path("D:/temp/", "01_ИСХОДНИК-НОВ (выгрузка 1С)_ТЕСТ.xlsx")
file_path_sales_plan = Path("D:/temp/_sales_plan/", "_current_sp.xlsx")
file_path_cumulative_copy = Path("D:/temp/__удалить/", t.strftime("%m%d%H%M%S") + "_01_ИСХОДНИК-НОВ (выгрузка 1С).xlsx")
file_path_sales_plan_copy = Path("D:/temp/__удалить/", t.strftime("%m%d%H%M%S") + "_current_sp.xlsx")

In [4]:
# создание архивных копий рабочих файлов
copyfile(file_path_cumulative, file_path_cumulative_copy)
copyfile(file_path_sales_plan, file_path_sales_plan_copy)

df_cf = pd.read_excel(file_path_cumulative, "TDSheet", usecols="A:O", index_col=False)
print("Записей в накопетельном файле: ", len(df_cf))

df_cf = df_cf.drop(df_cf.loc[df_cf['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Категория клиента (св-во Контрагент)'] == "VIP Чукурова"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Контрагент'] == "Колбико ООО"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Контрагент'] == "Укрдонмясо ООО"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Контрагент'] == "ПИР ООО"].index)
df_cf = df_cf.drop(df_cf.loc[df_cf['Контрагент'] == "ТАНДЕМ ООО ТК"].index)

print("Последняя запись в накопителе от ", df_cf["По дням"].max())

Записей в накопетельном файле:  143116
Последняя запись в накопителе от  2024-01-17 00:00:00


In [5]:
df_1c = YM.read_excel_from_1c(file_path_1c, header=7, usecols="B:P", index_col=False)
df_1c["По дням"] = pd.to_datetime(df_1c["По дням"], format='%d.%m.%Y')
print("Из 1с загружено строк: ", len(df_1c))

# обрезаем лишние пробелы 
df_obj = df_1c.select_dtypes(['object'])
df_1c[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

df_sp = pd.read_excel(file_path_sales_plan, header=7, usecols="B:P", index_col=False)
print("Последняя запись в накопителе sales_plan от ", df_sp["По дням"].max())

df_sp_new = pd.concat([df_sp,
                        df_1c.loc[df_1c["По дням"] > df_sp["По дням"].max()]
                        ])
                        
app_excel = xw.App(visible = True)
wb = xw.Book(file_path_sales_plan)
ws = wb.sheets['TDSheet']
ws.range('B8').options(pd.DataFrame, index=False).value = df_sp_new
wb.save()

df_1с = df_1c.rename(columns={
                    'Контрагент (категории)':'Категория клиента (св-во Контрагент)'
                    ,'Пользователь':'Основной менеджер покупателя'
                     })
df_1с["По дням"] = pd.to_datetime(df_1с["По дням"], format="%d.%m.%Y")
df_vip1 = df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"]
df_vip2 = df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"]
df_vip3 = df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Чукурова"]
df_kolbiko = df_1с.loc[df_1с['Контрагент'] == "Колбико ООО"]
df_urkdonmyaso = df_1с.loc[df_1с['Контрагент'] == "Укрдонмясо ООО"]
df_pir = df_1с.loc[df_1с['Контрагент'] == "ПИР ООО"]
df_mpp = df_1с.loc[df_1с['Контрагент'] == "ТАНДЕМ ООО ТК"]

df_vip = pd.concat([df_vip1, df_vip2, df_vip3, df_kolbiko, df_urkdonmyaso, df_pir, df_mpp])
df_1с_trunc = df_1с.drop(df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Прямые продажи"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "Дистрибьютор"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Категория клиента (св-во Контрагент)'] == "VIP Чукурова"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Контрагент'] == "Колбико ООО"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Контрагент'] == "Укрдонмясо ООО"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Контрагент'] == "ПИР ООО"].index)
df_1с_trunc = df_1с_trunc.drop(df_1с.loc[df_1с['Контрагент'] == "ТАНДЕМ ООО ТК"].index)
print("Последняя запись в базе 1с от ", df_1с_trunc["По дням"].max())

df_cf_new = pd.concat([df_cf,
                       df_vip,
                       df_1с_trunc.loc[df_1с_trunc["По дням"] > df_cf["По дням"].max()]
                      ])
df_cf_new = df_cf_new.dropna(subset = ["Количество (в ед. отчетов)"])
df_cf_new.loc[df_cf_new["Контрагент"] == "Конечный оптовый покупатель", ["Грузополучатель"]] = ""

app_excel = xw.App(visible = True)
wb = xw.Book(file_path_cumulative)
ws = wb.sheets['TDSheet']
ws.range('A1').options(pd.DataFrame, index=False).value = df_cf_new
wb.api.RefreshAll()
wb.save()

'''
wb.close()
app_excel.kill()
del app_excel
'''

print("Строк в новом файле: ", len(df_cf_new))
print("Время выполнения ", t.time() - ts1)

Файл перепакован из-за ошибки: "There is no item named 'xl/sharedStrings.xml' in the archive"
Из 1с загружено строк:  39810
Последняя запись в накопителе sales_plan от  2024-01-17 00:00:00
Последняя запись в базе 1с от  2024-01-18 00:00:00
Строк в новом файле:  39828
Время выполнения  109.73730230331421


In [6]:
# Проверка на непонятно зачем создаваемый фильтр _FilterDatabase
for n in wb.names:
    n.Visible = True
    print(n)
    
if wb.names.contains("TDSheet!_FilterDatabase"):
    wb.names("TDSheet!_FilterDatabase").delete()
    print("Фильтр TDSheet!_FilterDatabase удалён!")

if wb.names.contains("Бонусы!_FilterDatabase"):
    wb.names("Бонусы!_FilterDatabase").delete()
    print("Фильтр Бонусы!_FilterDatabase!")

if wb.names.contains("'Иванов Горловка'!_FilterDatabase"):
    wb.names("'Иванов Горловка'!_FilterDatabase").delete()
    print("Фильтр 'Иванов Горловка'!_FilterDatabase!")


<Name 'TDSheet!_FilterDatabase': =TDSheet!$A$1:$Z$143117>
<Name '_xlfn.AVERAGEIFS': =#NAME?>
<Name '_xlfn.IFERROR': =#NAME?>
<Name 'ee': ="">
<Name 'q': ="">
<Name 'w': ="">
<Name 'Срез_По_дням': ="">
Фильтр TDSheet!_FilterDatabase удалён!
